# Source Detection

This notebook shows an end to end radio interferometry pipeline from the sky to final image using the Karabo pipeline. In addition, we show you how to get the pixel coordinates of the point sources of the produced image. The simulation of the sky, telescope and observation are based on [OSKAR](https://ska-telescope.gitlab.io/sim/oskar/), the imaging on [RASCIL](https://ska-telescope.gitlab.io/external/rascil/index.html).

For this example we use the GLEAM survey [GLEAM EGC catalog version 2](https://vizier.cds.unistra.fr/viz-bin/VizieR-3), which can be downloaded from the [VizieR](https://cdsarc.unistra.fr/viz-bin/cat/VIII/100) service. To download the full survey, you have to change **preferences/max** to unlimited. The download here was done using the **FITS (binary) Table** option which can be selected in the dropdown menu below. The GLEAM survey was created using the Murchison Widefield Array (MWA), the low-frequency Square Kilometre Array (SKA1 LOW) precursor located in western Australia. The catalogue covers 24,402 square degrees, over declinations south of +30° and galactic latitudes outside 10° of the galactic plane, excluding some areas such as the Magellanic clouds. It contains 307,456 radio sources with 20 separate flux density measurements across 72-231 MHz, selected from a time- and frequency- integrated image centred at 200 MHz, with a resolution of ~=2'.

In [ ]:
import sys
import oskar
import matplotlib
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
import numpy as np

from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from astropy import wcs

## Sky Catalog

Of course, any catalog downloaded in any format such as .fits or .csv can be used. Since the structures of the catalogs can differ significantly, we leave the preprocessing to the format of the `SkyModel` to the user. The `SkyModel` has the following catalog format:

- right_ascension:
- declination:
- stokes_I_flux:
- stokes_Q_flux:
- stokes_U_flux:
- stokes_V_flux:
- reference_frequency:
- spectral_index:
- rotation_measure:
- major_axis_FWHM:
- minor_axis_FWHM:
- position_angle:
- source_id:

For this example we only use right ascension (RAJ2000) and declination (DEJ2000) and the Stokes I peak flux intensity at 76 MHz (Fp076). There are some sources which have no flux values in the corresponding frequency band. These must be removed first.

In [ ]:
import pandas as pd

df_racs = pd.read_csv('AS110_Derived_Catalogue_racs_dr1_gaussians_galacticregion_v2021_08_v01_4550.csv')
df_racs.head()

In [ ]:
df_racs.columns

In [ ]:
from datetime import timedelta, datetime
import sys
sys.path.insert(1, '/home/lukas/i4ds/ska/KaraboPipeline')
from karabo.simulation.sky_model import *
from karabo.simulation.observation import *
from karabo.simulation.utils import *
from karabo.simulation.telescope import *
from karabo.simulation.observation import *
from karabo.simulation.interferometer import *

In [ ]:
gleam = SkyModel.get_fits_catalog('./GLEAM_EGC.fits')
df_gleam = gleam.to_pandas()
df_gleam.head()

In [ ]:
ref_freq = 76e6
start_frequency_hz = 76e6
df_gleam = df_gleam[~df_gleam['Fp076'].isna()]
ra, dec, fp = df_gleam['RAJ2000'], df_gleam['DEJ2000'], df_gleam['Fp076']
sky_array = np.column_stack((ra, dec, fp, np.zeros(ra.shape[0]), np.zeros(ra.shape[0]), 
                             np.zeros(ra.shape[0]), [ref_freq]*ra.shape[0])).astype('float32')
sky = SkyModel(sky_array)
sky[:,-1] = df_gleam['GLEAM']
sky.num_sources

Before further steps are taken, the phase center of the telescope must be defined.

In [ ]:
ra0 = 250
dec0 = -80
phase_center = [ra0,dec0]

In [ ]:
def plot_sky(sky, phase_center):
    ra0, dec0 = phase_center[0], phase_center[1]
    data = sky[:,0:3]
    ra = np.radians(data[:, 0] - ra0)
    dec = np.radians(data[:, 1])
    flux = data[:, 2]
    log_flux = np.log10(flux) # doesn't work as intended yet because of log of negative values
    x = np.cos(dec) * np.sin(ra)
    y = np.cos(np.radians(dec0)) * np.sin(dec) - \
                np.sin(np.radians(dec0)) * np.cos(dec) * np.cos(ra)
    sc = plt.scatter(x, y, s=.5, c=log_flux, cmap='plasma',
                vmin=np.min(log_flux), vmax=np.max(log_flux))
    plt.axis('equal')
    plt.xlabel('x direction cosine')
    plt.ylabel('y direction cosine')
    plt.colorbar(sc, label='Log10(Stokes I flux [Jy])')
    plt.show()
    
plot_sky(sky, phase_center)

Now that we have determined the phase center we can take a closer look at how the point sources are distributed around the center. For this we use a coordinate 2d image projection with the phase center as the origin using the the world coordinate system ([wcs](https://docs.astropy.org/en/stable/wcs/index.html)) of astropy. Specific information about the wcs parameters can be found [here](https://docs.astropy.org/en/stable/api/astropy.wcs.Wcsprm.html). For this illustration, we do not take into account the flux intensity, so that all sources are clearly visible.

In [ ]:
sky.explore_sky(phase_center=phase_center, figsize=(8,6))

Now, to have only a partition of the sky, we can use the `filter_by_radius`, which filters from the phase center with an inner and outer radius in degrees. Oskar also provides the function `filter_by_flux` to filter by Stokes-I flux. For more details we refer to the Oskar [documentation](https://fdulwich.github.io/oskarpy-doc/sky.html).

In [ ]:
# first filter sky by inner and outer radius in degrees from the phase center
sky.filter_by_radius(0, .55, phase_center[0], phase_center[1])
sky.num_sources

Now let's look at the filtered sources.

In [ ]:
sky.explore_sky(phase_center=phase_center, figsize=(10,8), s=80,
                xlim=(-.55,.55), ylim=(-.55,.55), with_labels=True)

### Telescope Module

Various observation parameters and meta information `params` must be passed to the telescope module `oskar.Interferometer` of OSKAR as `oskar.SettingsTree`.

In [ ]:
telescope = read_OSKAR_tm_file('../data/telescope.tm')

In [ ]:
observation = Observation(start_frequency_hz=start_frequency_hz, start_date_and_time=datetime(2000, 1, 1, 12, 0),
                          length=timedelta(hours=12), number_of_channels=16, frequency_increment_hz=20e6,
                          phase_centre_ra_deg=ra0, phase_centre_dec_deg=dec0, number_of_time_steps=24)

In [ ]:
interferometer = InterferometerSimulation(ms_path='./visibilities_gleam.ms', channel_bandwidth_hz=1e6,
                                          time_average_sec=10)

In [ ]:
interferometer.run_simulation(telescope, sky, observation)

In [ ]:
#os_sky = sky.get_OSKAR_sky()

# Basic settings. (Note that the sky model is set up later.)
#params = {
#    "simulator": {
#        "use_gpus": "False"
#    },
#    "observation" : {
#        #"num_channels": 64,
#        "num_channels": "16",
#        "start_frequency_hz": str(start_frequency_hz),
#        "frequency_inc_hz": "20e6",
#        "phase_centre_ra_deg": str(phase_center[0]),
#        "phase_centre_dec_deg": str(phase_center[1]),
#        "num_time_steps": "24",
#        "start_time_utc": "01-01-2000 12:00:00.000",
#        "length": "12:00:00.000"
#    },
#    "telescope": {
#        "input_directory": "../data/telescope.tm"
#    },
#    "interferometer": {
#        "ms_filename": "visibilities_gleam.ms",
#        "channel_bandwidth_hz": str(1e6),
#        "time_average_sec": "10"
#    }
#}
#settings = oskar.SettingsTree("oskar_sim_interferometer")
#settings.from_dict(params)

#precision = "single"
#if precision == "single":
#    settings["simulator/double_precision"] = "False"
#    settings["simulator/double_precision"] = "True"

# Set the sky model and run the simulation.
#sim = oskar.Interferometer(settings=settings)
#sim.set_sky_model(os_sky)
#sim.run()

### Observation Simulation

Now the sky module must be passed to the interferometer and the simulation of the observation must be started to generate the measurement set.

## Processing

After the observation is made with the telescope, a calibration of the measured data must be performed, followed by the reconstruction of the image.

### Calibration after Observation

toDo

### Imaging

Start an mmclean algorithm with the visibilites.ms as an input to deconvolve. 
To use dask cluster where you can see the progress, first create a dask cluster in the dask-extension on the left. 
Then copy the scheduler adress into the variable below. It might be correct already.

If you don't do this, remove the --dask_scheduler option from the options in the start_imager call.
Then RASCIL starts its own scheduler, you will however not be able to see the dashbaord, as the port is probably not forwarded by docker.

In [ ]:
#from rascil.apps import rascil_imager
#from rascil.processing_components.util.performance import (
#    performance_store_dict,
#    performance_environment,
#)
    
#def start_imager(rawargs):
#    parser = rascil_imager.cli_parser()
#    args = parser.parse_args(rawargs)
#    performance_environment(args.performance_file, mode="w")
#    performance_store_dict(args.performance_file, "cli_args", vars(args), mode="a")
#    image_name = rascil_imager.imager(args)

#start_imager(
#    [
#        '--ingest_msname','visibilities_gleam.ms',
#        '--ingest_dd', '0', 
#        '--ingest_vis_nchan', '16',
#        '--ingest_chan_per_blockvis', '1' ,
#        '--ingest_average_blockvis', 'True',
#        '--imaging_npixel', '2048', 
#        '--imaging_cellsize', '3.878509448876288e-05',
#        '--imaging_weighting', 'robust',
#        '--imaging_robustness', '-0.5',
#        '--clean_nmajor', '2' ,
#        '--clean_algorithm', 'mmclean',
#        '--clean_scales', '0', '6', '10', '30', '60',
#        '--clean_fractional_threshold', '0.3',
#        '--clean_threshold', '0.12e-3',
#        '--clean_nmoment' ,'5',
#        '--clean_psf_support', '640',
#        '--clean_restored_output', 'integrated'
#    ])

In [ ]:
from karabo.simulation.imager import *

imager = Imager(ingest_msname='./visibilities_gleam.ms',
                ingest_dd=[0],
                ingest_vis_nchan=16,
                ingest_chan_per_blockvis=1,
                ingest_average_blockvis=True,
                imaging_npixel=2048,
                imaging_cellsize=3.878509448876288e-05,
                imaging_weighting='robust',
                imaging_robustness=-.5,
                clean_nmajor=2,
                clean_algorithm='mmclean',
                clean_scales=[0,6,10,30,60],
                clean_fractional_threshold=.3,
                clean_threshold=.12e-3,
                clean_nmoment=5,
                clean_psf_support=640,
                clean_restored_output='integrated')

imager.imaging_rascil()

## Analysis and Comparison

toDo

Now we have the image of the pipeline and the corresponding pixel coordinates as ground truth. Important are the number of set pixels and the cellsize, from which the coordinates are calculated.

In [ ]:
from karabo.simulation.imager import *

px, py, sky_idxs = Imager.get_pixel_coord(imaging_cellsize=3.878509448876288e-05, imaging_npixel=2048,
                                          sky=sky, filter_outlier=True)
fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(px, py)
plt.xlim((0,2048))
plt.ylim((0,2048))

for i, txt in enumerate(sky[sky_idxs,-1]):
    #ax.annotate(txt, (px[i], py[i]))
    pass
plt.show()

In [ ]:
pd.DataFrame({'sources':sky[sky_idxs,-1],'px':px,'py':py})